In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
import copy

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns",None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
df = pd.read_csv("football_train.csv")

In [ ]:
df.head()

In [ ]:
cor = df.drop(["rating_num","scout_id","team1_system_id","team2_system_id","competitionId","player_position_1","player_position_2"],axis=1).corr()
l=[]
drop=[]
for i in range(cor.shape[0]):
    for j in range(i):
        if (abs(cor.iloc[i,j])>0.8 and abs(cor.iloc[i,j])!=1):
            if(abs(df[cor.columns[j]].corr(df["rating_num"]))>abs(df[cor.index[i]].corr(df["rating_num"]))):
                print("Drop",cor.index[i])
                drop.append(cor.index[i])
            else:
                print("Drop",cor.columns[j])
                drop.append(cor.columns[j])
            l.append([cor.columns[j],cor.index[i]])

In [ ]:
drop = list(set(drop))

In [ ]:
df.drop(drop,axis=1,inplace=True)

In [ ]:
pd.DataFrame(df.isnull().sum()).transpose()

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.dropna(how='all', axis=0, inplace=True)
df.shape

In [ ]:
df = pd.get_dummies(df,columns=["team"],drop_first=True)

In [ ]:
df.head()

In [ ]:
df = df[df["player_position_1"].notnull()]
df = df[df["player_height"].notnull()]
df = df[df["player_weight"].notnull()]

In [ ]:
pd.DataFrame(df.isnull().sum()).transpose()

In [ ]:
label_encoder = LabelEncoder()
df['winner']= label_encoder.fit_transform(df['winner'])

In [ ]:
x=df.drop(["scout_id","winner","team_team2","team1_system_id","team2_system_id","competitionId","rating_num"],axis=1)
y=df["rating_num"]

In [ ]:
x.fillna(0,inplace=True)

In [ ]:
x.isnull().sum()

In [ ]:
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
imp = model.feature_importances_

In [ ]:
imp

In [ ]:
plt.figure(figsize = (30,10))
feat = pd.Series(imp,index = x.columns)
feat.nlargest(100).plot.bar()

In [ ]:
feat.nlargest(100).index

In [ ]:
x_new = x[feat.nlargest(100).index]

In [ ]:
x_new

In [ ]:
scaler = StandardScaler()
x_new[x_new.columns] = scaler.fit_transform(x_new[x_new.columns])
x_train,x_test,y_train,y_test = train_test_split(x_new,y,test_size=0.25,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
r2_score(y_test, lr.predict(x_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rr = RandomForestRegressor()
rr.fit(x_train, y_train)
r2_score(y_test, rr.predict(x_test))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dr = DecisionTreeRegressor()
dr.fit(x_train, y_train)
r2_score(y_test, dr.predict(x_test))